<a href="https://colab.research.google.com/github/damladmrk/GeneticAlgorithms/blob/main/GAScheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Problem Definition

We are working on a simplified **University Course Timetabling Problem (UCTP)**.  
The goal is to assign:
- Courses → to teachers, rooms, and time slots.  

🔸 Each course must be taught by **one teacher** (from a predefined list of possible teachers).  
🔸 Each course must occupy **exactly one room** and **exactly one time slot**.  

We distinguish between:
- **Hard constraints** (must never be violated):  
  - A teacher cannot teach more than one course at the same time.  
  - A room cannot host more than one course at the same time.  
  - Each course is assigned to exactly one teacher, room, and slot.  

- **Soft constraints** (desirable but not mandatory):  
  - Avoid back-to-back classes for the same teacher.  
  - Try to balance sessions across days.  

The Genetic Algorithm (GA) will be used to **search for good schedules** that satisfy these rules as much as possible.


In [18]:
import pandas as pd
import json, pathlib
import random
import numpy as np
import matplotlib as plt

# 📊 Data Preparation

We define four key datasets:
- **Teachers**: A list of available instructors.  
- **Rooms**: Lecture halls and labs where courses can be scheduled.  
- **Slots**: Time slots (e.g., Mon 09:00–09:50).  
- **Courses**: Each course has:
    - A unique ID and name
    - A duration (number of slots required)
    - A list of possible teachers who can teach it  

These datasets form the **search space** for the algorithm.


In [2]:
# Teachers  (8)
teachers = pd.DataFrame({
    "teacher_id": [f"T{i}" for i in range(1,9)],
    "name": [
        "Alice Johnson", "Bob Smith", "Charlie Brown", "Diana Miller",
        "Edward Wilson", "Fiona Davis", "George Clark", "Helen Thompson"
    ]
})

# Rooms (6)
rooms = pd.DataFrame({
    "room_id": [f"R{i}" for i in range(1,7)],
    "name": ["101", "102", "201", "202", "301", "302"]
})

# Slots (12)
slots = pd.DataFrame({
    "slot_id": [f"S{i}" for i in range(1,13)],
    "day": ["Mon","Mon","Mon","Mon","Tue","Tue","Tue","Tue","Wed","Wed","Wed","Wed"],
    "time_block": [
        "09:00-09:50","10:00-10:50","11:00-11:50","13:00-13:50",
        "09:00-09:50","10:00-10:50","11:00-11:50","13:00-13:50",
        "09:00-09:50","10:00-10:50","11:00-11:50","13:00-13:50"
    ]
})

# Courses (12)
courses = pd.DataFrame({
    "course_id": [f"C{i}" for i in range(1,13)],
    "course_name": [
        "Algorithms","Linear Algebra","Intro to CS","Discrete Math","Data Structures","Physics I",
        "Programming Lab","Databases","Operating Systems","Computer Networks","Software Engineering",
        "Artificial Intelligence"
    ],
    "duration_slots": [1]*12,
    "possible_teachers": [
        ["T1","T2"],["T2","T3"],["T1","T4"],["T3","T4"],["T1","T2","T3"],["T2"],
        ["T1","T4"],["T5","T6"],["T6","T7"],["T7","T8"],["T5","T8"],["T4","T6","T7"]
    ]
})

constraints = {
        "hard": [
            "A teacher cannot teach more than one course in the same slot.",
            "A room cannot host more than one course in the same slot.",
            "Each course is assigned to exactly one teacher from its possible_teachers list.",
            "Each course occupies exactly one slot."
        ],
        "soft": [
            "Avoid back-to-back sessions for the same teacher if possible."
        ]
    }

# Save CSVs
teachers.to_csv("teachers_v3.csv", index=False)
rooms.to_csv("rooms_v3.csv", index=False)
slots.to_csv("slots_v3.csv", index=False)
courses.to_csv("courses_v3.csv", index=False)

# Display for Colab-style viewing
print("=== Teachers ===\n")
print(teachers)
print("\n=== Rooms ===\n")
print(rooms)
print("\n=== Slots ===\n")
print(slots)
print("\n=== Courses with Possible Teachers ===\n")
print(courses)
print("\n=== Constraints ===\n")
print(json.dumps(constraints, indent=2))

=== Teachers ===

  teacher_id            name
0         T1   Alice Johnson
1         T2       Bob Smith
2         T3   Charlie Brown
3         T4    Diana Miller
4         T5   Edward Wilson
5         T6     Fiona Davis
6         T7    George Clark
7         T8  Helen Thompson

=== Rooms ===

  room_id name
0      R1  101
1      R2  102
2      R3  201
3      R4  202
4      R5  301
5      R6  302

=== Slots ===

   slot_id  day   time_block
0       S1  Mon  09:00-09:50
1       S2  Mon  10:00-10:50
2       S3  Mon  11:00-11:50
3       S4  Mon  13:00-13:50
4       S5  Tue  09:00-09:50
5       S6  Tue  10:00-10:50
6       S7  Tue  11:00-11:50
7       S8  Tue  13:00-13:50
8       S9  Wed  09:00-09:50
9      S10  Wed  10:00-10:50
10     S11  Wed  11:00-11:50
11     S12  Wed  13:00-13:50

=== Courses with Possible Teachers ===

   course_id              course_name  duration_slots possible_teachers
0         C1               Algorithms               1          [T1, T2]
1         C2          

# 🧬 Chromosome Representation

In the GA, each **chromosome** represents a possible weekly schedule.

- Each chromosome is a **list of 12 genes** (since we have 12 courses).  
- Each gene corresponds to a single course and encodes:
  - `teacher_id`
  - `room_id`
  - `slot_id`

Example gene:  
`("T3", "R2", "S7")` → Course is taught by Teacher T3, in Room R2, at Slot S7.  

The order of genes corresponds to the course list:
- Gene 1 → Course C1
- Gene 2 → Course C2
- ...
- Gene 12 → Course C12


In [3]:
def generate_population(courses, teachers, rooms, slots, population_size=30):
    """
    Generate an initial random population for the timetabling problem.

    Each chromosome is a list of genes.
    Each gene corresponds to one course and contains:
        - teacher_id (chosen from course's possible_teachers list)
        - room_id (random from available rooms)
        - slot_id (random from available slots)

    NOTE:
    - This is just a random initialization (likely violates constraints).
    - Genetic Algorithm will improve these solutions through selection, crossover, and mutation.
    """

    population = []

    for _ in range(population_size):
        chromosome = []
        for _, course in courses.iterrows():
            teacher = random.choice(course["possible_teachers"])
            room = random.choice(rooms["room_id"].tolist())
            slot = random.choice(slots["slot_id"].tolist())

            # Each gene = (teacher, room, slot)
            chromosome.append((teacher, room, slot))

        population.append(chromosome)

    return population

In [4]:
population = generate_population(courses, teachers, rooms, slots, population_size=30)

# Print first chromosome for inspection
print("First chromosome (example):\n")
for i, gene in enumerate(population[0], start=1):
    print(f"Course C{i}: Teacher={gene[0]}, Room={gene[1]}, Slot={gene[2]}")

First chromosome (example):

Course C1: Teacher=T2, Room=R2, Slot=S6
Course C2: Teacher=T2, Room=R4, Slot=S2
Course C3: Teacher=T4, Room=R2, Slot=S10
Course C4: Teacher=T3, Room=R2, Slot=S10
Course C5: Teacher=T3, Room=R3, Slot=S9
Course C6: Teacher=T2, Room=R4, Slot=S9
Course C7: Teacher=T1, Room=R1, Slot=S4
Course C8: Teacher=T6, Room=R1, Slot=S7
Course C9: Teacher=T7, Room=R5, Slot=S10
Course C10: Teacher=T8, Room=R6, Slot=S10
Course C11: Teacher=T8, Room=R5, Slot=S4
Course C12: Teacher=T6, Room=R4, Slot=S9


### Fitness Function Approaches

In our timetabling problem, the **fitness function** is the key evaluation metric that tells us how good a solution (chromosome) is. Since we can design fitness functions in multiple ways, here we experiment with **two different approaches.** By comparing the two, we can **observe how different fitness scaling affects the behavior of the Genetic Algorithm (GA)**.  :

---

#### 1. Linear Fitness  
- Formula:  
  `fitness = K - penalty`  
- Idea: Start with a high score (e.g., 4000) and subtract penalties for every violation of hard or soft constraints.  
- Pros:  
  - Keeps a wide range of fitness values, making it easier to differentiate between solutions.  
  - Even “bad” solutions still get nonzero scores, which helps in selection.  
- Cons:  
  - Requires choosing a proper `max_score`. If it’s too low, many solutions will drop to zero; if it’s too high, penalties might look too small.  

---

#### 2. Inverse Fitness  
- Formula:  
  `fitness = 1 / (1 + penalty)`  
- Idea: Fitness decreases inversely with the penalty. If the penalty grows large, fitness quickly approaches zero.  
- Pros:  
  - Strongly punishes invalid or poor solutions (they almost collapse to 0).  
  - Guarantees a bounded range: always between (0, 1].  
- Cons:  
  - Bad solutions are hard to differentiate (most values cluster near zero).  



In [5]:
def fitness_inverse(chromosome, courses, slots):
    """
    Calculate fitness of a given chromosome.
    Lower penalty = better fitness.
    """
    penalty = 0

    # Build helper structures
    teacher_slot = {}
    room_slot = {}

    for i, gene in enumerate(chromosome):
        teacher, room, slot = gene
        course = courses.iloc[i]

        # --- HARD CONSTRAINTS ---
        # Teacher cannot teach >1 course in same slot
        if (teacher, slot) in teacher_slot:
            penalty += 1000
        else:
            teacher_slot[(teacher, slot)] = course["course_id"]

        # Room cannot host >1 course in same slot
        if (room, slot) in room_slot:
            penalty += 1000
        else:
            room_slot[(room, slot)] = course["course_id"]

        # Teacher must be in possible_teachers
        if teacher not in course["possible_teachers"]:
            penalty += 1000

    # --- SOFT CONSTRAINTS ---
    # Avoid back-to-back sessions for the same teacher
    slot_order = {sid: idx for idx, sid in enumerate(slots["slot_id"].tolist())}
    teacher_slots = {}

    for i, gene in enumerate(chromosome):
        teacher, _, slot = gene
        slot_idx = slot_order[slot]
        if teacher not in teacher_slots:
            teacher_slots[teacher] = []
        teacher_slots[teacher].append(slot_idx)

    for teacher, slot_indices in teacher_slots.items():
        slot_indices.sort()
        for j in range(len(slot_indices) - 1):
            if slot_indices[j+1] == slot_indices[j] + 1:
                penalty += 10  # small penalty

    # --- FITNESS VALUE ---
    fitness = 1 / (1 + penalty)  # Higher is better
    return fitness


In [6]:
def fitness_lineer(chromosome, courses, slots):
    """
    Calculate fitness of a given chromosome.
    Lower penalty = better fitness.
    """
    penalty = 0

    # Build helper structures
    teacher_slot = {}
    room_slot = {}

    for i, gene in enumerate(chromosome):
        teacher, room, slot = gene
        course = courses.iloc[i]

        # --- HARD CONSTRAINTS ---
        # Teacher cannot teach >1 course in same slot
        if (teacher, slot) in teacher_slot:
            penalty += 1000
        else:
            teacher_slot[(teacher, slot)] = course["course_id"]

        # Room cannot host >1 course in same slot
        if (room, slot) in room_slot:
            penalty += 1000
        else:
            room_slot[(room, slot)] = course["course_id"]

        # Teacher must be in possible_teachers
        if teacher not in course["possible_teachers"]:
            penalty += 1000

    # --- SOFT CONSTRAINTS ---
    # Avoid back-to-back sessions for the same teacher
    slot_order = {sid: idx for idx, sid in enumerate(slots["slot_id"].tolist())}
    teacher_slots = {}

    for i, gene in enumerate(chromosome):
        teacher, _, slot = gene
        slot_idx = slot_order[slot]
        if teacher not in teacher_slots:
            teacher_slots[teacher] = []
        teacher_slots[teacher].append(slot_idx)

    for teacher, slot_indices in teacher_slots.items():
        slot_indices.sort()
        for j in range(len(slot_indices) - 1):
            if slot_indices[j+1] == slot_indices[j] + 1:
                penalty += 10  # small penalty

    # --- FITNESS VALUE ---
    fitness = 4000 - penalty  # Higher is better
    return fitness


In [20]:
def evaluate_population(population, courses, slots, fitness_fn):
    return [fitness_fn(ch, courses, slots) for ch in population]


In [12]:
def select_parents(population, fitness_values, k=4):
    parents = []
    for _ in range(2):
        cand_idx = random.sample(range(len(population)), k)
        best_idx = max(cand_idx, key=lambda idx: fitness_values[idx])
        parents.append(population[best_idx])
    return parents[0], parents[1]

In [16]:
def crossover_uniform(parent1, parent2, crossover_rate=0.9, gene_swap_prob=0.5):
    if random.random() >= crossover_rate:
        return parent1.copy(), parent2.copy()

    child1, child2 = [], []
    for g1, g2 in zip(parent1, parent2):
        if random.random() < gene_swap_prob:
            child1.append(g1)
            child2.append(g2)
        else:
            child1.append(g2)
            child2.append(g1)
    return child1, child2



In [14]:
def mutate(individual, rooms, slots, courses, mutation_rate=0.1):
    new_individual = individual.copy()
    for i in range(len(new_individual)):
        if random.random() < mutation_rate:
            teacher, room, slot = new_individual[i]
            course = courses.iloc[i]

            mtype = random.choice(["teacher", "room", "slot"])
            if mtype == "teacher":
                teacher = random.choice(course["possible_teachers"])
            elif mtype == "room":
                room = random.choice(rooms["room_id"].tolist())
            else:  # slot
                slot = random.choice(slots["slot_id"].tolist())

            new_individual[i] = (teacher, room, slot)
    return new_individual


In [17]:
def create_new_population(population, courses, slots, rooms, fitness_fn,
                          crossover_rate=0.9, mutation_rate=0.1,
                          tournament_k=4, elite_size=2, gene_swap_prob=0.5):
    fitness_values = evaluate_population(population, courses, slots, fitness_fn)

    ranked_idx = sorted(range(len(population)),
                        key=lambda i: fitness_values[i], reverse=True)
    new_population = [population[i].copy() for i in ranked_idx[:elite_size]]

    while len(new_population) < len(population):
        p1, p2 = select_parents(population, fitness_values, k=tournament_k)
        c1, c2 = crossover_uniform(p1, p2, crossover_rate=crossover_rate, gene_swap_prob=gene_swap_prob)
        c1 = mutate(c1, rooms=rooms, slots=slots, courses=courses, mutation_rate=mutation_rate)
        new_population.append(c1)
        if len(new_population) < len(population):
            c2 = mutate(c2, rooms=rooms, slots=slots, courses=courses, mutation_rate=mutation_rate)
            new_population.append(c2)

    return new_population

In [22]:
# Genetic Algorithm Execution
def run_genetic_algorithm(courses, teachers, rooms, slots, generations=50, population_size=20):
    print("=== Starting Genetic Algorithm for Timetabling ===")
    print(f"Population size: {population_size}")
    print(f"Number of generations: {generations}")
    print(f"Number of courses: {len(courses)}")
    print(f"Number of teachers: {len(teachers)}")
    print(f"Number of rooms: {len(rooms)}")
    print(f"Number of time slots: {len(slots)}")
    print()

    # Generate initial population
    population = generate_population(courses, teachers, rooms, slots, population_size)

    # Track best fitness over generations
    best_fitness_history = []
    avg_fitness_history = []

    for generation in range(generations):
        # Evaluate population
        fitness_values = evaluate_population(population, courses, slots, fitness_lineer)

        # Calculate statistics
        best_fitness = max(fitness_values)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        best_fitness_history.append(best_fitness)
        avg_fitness_history.append(avg_fitness)

        # Find best individual
        best_idx = fitness_values.index(best_fitness)
        best_individual = population[best_idx]

        # Print progress every 10 generations
        if generation % 5 == 0 or generation == generations - 1:
            print(f"Generation {generation}: Best Fitness = {best_fitness:.2f}, Avg Fitness = {avg_fitness:.2f}")

            # Count constraint violations for best individual
            penalty = 4000 - best_fitness  # Since we're using fitness_lineer
            print(f"  Constraint violations: {penalty} penalty points")

            if penalty == 0:
                print("  ✅ Perfect solution found!")

        # Create new population
        population = create_new_population(
            population, courses, slots, rooms, fitness_lineer,
            crossover_rate=0.8, mutation_rate=0.15,
            tournament_k=3, elite_size=2
        )

    # Final evaluation
    fitness_values = evaluate_population(population, courses, slots, fitness_lineer)
    best_fitness = max(fitness_values)
    best_idx = fitness_values.index(best_fitness)
    best_individual = population[best_idx]

    print("\n=== Final Results ===")
    print(f"Best fitness achieved: {best_fitness:.2f}")

    # Display the best schedule
    print("\n=== Best Schedule ===")
    for i, gene in enumerate(best_individual):
        course = courses.iloc[i]
        print(f"Course {course['course_id']} ({course['course_name']}):")
        print(f"  Teacher: {gene[0]}, Room: {gene[1]}, Slot: {gene[2]}")

    return best_individual, best_fitness_history, avg_fitness_history

# Run the genetic algorithm
best_schedule, best_history, avg_history = run_genetic_algorithm(
    courses, teachers, rooms, slots,
    generations=100, population_size=50
)


=== Starting Genetic Algorithm for Timetabling ===
Population size: 50
Number of generations: 100
Number of courses: 12
Number of teachers: 8
Number of rooms: 6
Number of time slots: 12

Generation 0: Best Fitness = 4000.00, Avg Fitness = 2690.20
  Constraint violations: 0 penalty points
  ✅ Perfect solution found!
Generation 5: Best Fitness = 4000.00, Avg Fitness = 3235.60
  Constraint violations: 0 penalty points
  ✅ Perfect solution found!
Generation 10: Best Fitness = 4000.00, Avg Fitness = 3337.60
  Constraint violations: 0 penalty points
  ✅ Perfect solution found!
Generation 15: Best Fitness = 4000.00, Avg Fitness = 3536.60
  Constraint violations: 0 penalty points
  ✅ Perfect solution found!
Generation 20: Best Fitness = 4000.00, Avg Fitness = 3713.40
  Constraint violations: 0 penalty points
  ✅ Perfect solution found!
Generation 25: Best Fitness = 4000.00, Avg Fitness = 3435.80
  Constraint violations: 0 penalty points
  ✅ Perfect solution found!
Generation 30: Best Fitness =